# 1st option

In [1]:
# 1st stage - importing libraries and data

import requests
import pandas as pd 
import datetime
import time

start = time.time()     # Starting time of the program for measurement purposes

response1 = requests.get("https://restcountries.eu/rest/v2/region/europe?fields=name;nativeName;alpha2Code;alpha3Code;latlng;population;area;capital;languages;topLevelDomain;timezones;regionalBlocs;currencies")

print(f"Status code is: {response1.status_code}")

eu_json = response1.json()

from math import sin, cos, sqrt, atan2, radians


def distance_pl(latlon):    # Function which calculates the distance between x country and Poland
    
    R = 6373.0

    lat1 = radians(52.0)
    lon1 = radians(20.0)
    lat2 = radians(latlon[0])
    lon2 = radians(latlon[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return round(distance,3)



from coinbase.wallet.client import Client

client = Client("https://api.coinbase.com/v2/exchange-rates",api_secret="PLN")

# 2nd Stage - Looping per country and appending lists (yeah, I know...)

name=[]
nativeName=[]
alpha2Code=[]
alpha3Code=[]
latlng=[]
population=[]
area=[]
capital=[]
languages=[]
topLevelDomain=[]
timezones=[]
regionalBlocs=[]
currencies=[]
currencies_symbol=[]
currencies_to_PLN=[]
distance_to_PL=[]

print("Uploading and clearing data...")

for x in eu_json:
    
    to_PLN = client.get_exchange_rates(currency=x["currencies"][0]["code"])
    
    name.append(x["name"])
    nativeName.append(x["nativeName"])
    alpha2Code.append(x["alpha2Code"])
    alpha3Code.append(x["alpha3Code"])
    latlng.append(x["latlng"])
    population.append(x["population"])
    try:
        area.append(x["area"])
    except KeyError:
        area.append("")
    capital.append(x["capital"])
    whole_str=""
    for lng in x["languages"]:
        if len(x["languages"]) <2:
            lng_name= lng["name"]
            whole_str = whole_str + str(f"{lng_name}, ")
        else:
            lng_name = lng["name"]
            whole_str = whole_str + str(f"{lng_name}, ")
    
    languages.append(whole_str[:-2])
    topLevelDomain.append(x["topLevelDomain"][0])
    timezones.append(x["timezones"])
    if x["regionalBlocs"]==[]:
        regionalBlocs.append(False)
    elif "EU" not in x["regionalBlocs"][0].values() and "European Union" not in x["regionalBlocs"][0].values():
        regionalBlocs.append(False)
    else:
        regionalBlocs.append(True)
    currencies.append(x["currencies"][0]["name"])
    if "symbol" in x["currencies"][0]:
        currencies_symbol.append(x["currencies"][0]["symbol"])
    else:
        currencies_symbol.append("")
    currencies_to_PLN.append(round(float(to_PLN["rates"]["PLN"]),2))
    distance_to_PL.append(distance_pl(x["latlng"]))



# 3rd stage - making DataFrame and dumping it to csv file

pd.DataFrame({"Country name - English":name,"Country name - Native":nativeName,"Country 2 code":alpha2Code,"Country 3 code":alpha3Code,"Coordinates":latlng,"Population":population,"Area":area,"Capital city":capital,"Languages":languages,"topLevelDomain":topLevelDomain,"timezones":timezones,"Is the country in EU?":regionalBlocs,"Distance to Poland in km":distance_to_PL,"Currency name":currencies,"Currency symbol":currencies_symbol,"To PLN":currencies_to_PLN,"Time of extraction":datetime.datetime.now() }).to_csv("raport1.csv")

end = time.time()     # Ending time of the program for measurement purposes
print(f"Done! It took me {round(end - start,2)} seconds")

Status code is: 200
Uploading and clearing data...
Done! It took me 6.95 seconds


# 2nd option

In [2]:
# 1st stage - importing data

import requests
import pandas as pd 
import json
import time
from coinbase.wallet.client import Client

start = time.time()     # Starting time for measurement purposes


response1 = requests.get("https://restcountries.eu/rest/v2/region/europe?fields=name;nativeName;alpha2Code;alpha3Code;latlng;population;area;capital;languages;topLevelDomain;timezones;regionalBlocs;currencies")

print(f"Status code is: {response1.status_code}")  # Printing status code

eu_json = response1.json()

df = pd.read_json(json.dumps(eu_json))      # df as a DataFrame from JSON type data


# 2nd stage - time for rearranging the df

cols =["name","nativeName","alpha2Code","alpha3Code","latlng","population","area","capital","languages","topLevelDomain","timezones","regionalBlocs","currencies"]
df = df[cols]       # Changing order of the columns

# 3rd stage - defining some useful functions

print("Organizing data...")

def languages_correct(x):       # Function which transforms "languages" data into more lucid data
    whole_str=""
    for lng in x:
        if len(x) <2:
            lng_name= lng["name"]
            whole_str = whole_str + str(f"{lng_name}, ")
        else:
            lng_name = lng["name"]
            whole_str = whole_str + str(f"{lng_name}, ")
    return whole_str[:-2]
    
df["languages"] = df["languages"].apply(languages_correct)      # Applying the function for the df column




from math import sin, cos, sqrt, atan2, radians


def distance_pl(latlon):    # Function which calculates distance between some country and Poland
    
    R = 6373.0

    lat1 = radians(52.0)    # latitiude of Poland
    lon1 = radians(20.0)        # Lenght of Poland 
    lat2 = radians(latlon[0])   # Latitiude of x Country
    lon2 = radians(latlon[1])   #Lenght of x Country

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return round(distance,3)    # Returns distance in km between two points

df["Distance from PL in km"] = df["latlng"].apply(distance_pl)  # Applying the function for the df column




def in_eu(x):       # Function which returns if the Country is in EU or not
    if len(x)==1:
        if x[0]["acronym"]=="EU":
            return True
        else:
            return False
    else:
        return False
df["regionalBlocs"] = df["regionalBlocs"].apply(in_eu)      # Applying the function for the df column
df = df.rename(columns={"regionalBlocs":"Is the country in EU?"})       # Changing the name of the column ( just for a better look :) )




def domain_correct(x):      # Function which clears output of the domain of the country
    if len(x)>0:
        return x[0]
    else:
        return ""
df["topLevelDomain"] = df["topLevelDomain"].apply(domain_correct)       # Applying the function for the df column
df = df.rename(columns={"topLevelDomain":"Country web domain"})       # Changing the name of the column ( just for a better look :) )





def currencies_name(x):         # Function which clarifies the currency data of the country
    return x[0]["name"]
def currencies_symbol(x):
    if "symbol" in x[0]:
        return x[0]["symbol"]
    else:
        return ""

df["currencies_symbol"] = df["currencies"].apply(currencies_symbol)       # Applying the function for the df column
df["currencies_name"] = df["currencies"].apply(currencies_name)       # Applying the function for the df column

print("Uploading exchange rates...")

def convert_to_pln(x):       # Function which uses given API to convert given currency to PLN
    client = Client("https://api.coinbase.com/v2/exchange-rates",api_secret="PLN")

    to_PLN = client.get_exchange_rates(currency=x[0]["code"])
    return round(float(to_PLN["rates"]["PLN"]),2)

df["To PLN"] = df["currencies"].apply(convert_to_pln)       # Applying the function for the df column
del df["currencies"]       # Deleting currencies column (we don't need it anymore)




import datetime

df["time of extraction"] = datetime.datetime.now()      # Making a column which contains current time of data extraction

#4th stage - creating a csv file

df.to_csv("raport2.csv")      # Finally making a csv file raport_eu.csv

end = time.time()       # Ending time for measurement purposes

print(f"Done! It took me {round(end-start,2)} seconds")     # Printing duration of the function

Status code is: 200
Organizing data...
Uploading exchange rates...
Done! It took me 9.13 seconds


# 3rd option

In [1]:
import json
import requests
from coinbase.wallet.client import Client
import datetime
from math import sin, cos, sqrt, atan2, radians
import csv
import sys
import time

start = time.time()     # Starting time for measurement purposes

print("Connecting...")

response1 = requests.get("https://restcountries.eu/rest/v2/region/europe?fields=name;nativeName;alpha2Code;alpha3Code;latlng;population;area;capital;languages;topLevelDomain;timezones;regionalBlocs;currencies")

if response1.status_code == 200:
    print(f"Connected! Status code is: {response1.status_code}")  # Printing when status code == 200 
else:
    print(f"Unable to connect. Check status code {response1.status_code}")   # Printing when status code != 200
    sys.exit()

eu_json = response1.json()  # Converting repsonse to JSON



def distance_pl(latlon):    # Function which calculates distance between some country and Poland
    
    R = 6373.0

    lat1 = radians(52.0)    # latitiude of Poland
    lon1 = radians(20.0)        # Lenght of Poland 
    lat2 = radians(latlon[0])   # Latitiude of x Country
    lon2 = radians(latlon[1])   #Lenght of x Country

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return round(distance,3)

client = Client("https://api.coinbase.com/v2/exchange-rates",api_secret="PLN")      # Connecting to exchange rates API

exchange_rates_dict={}      # Dict created for optimalization purposes

class Country:      # Creating Class for every country
    def __init__(self, name, nativeName,alpha2Code,alpha3Code,topLevelDomain,capital,population,latlng,timezones,languages,regionalBlocs,currencies,area=None):
        
        
        self.name = name
        self.nativeName = nativeName
        self.alpha2Code = alpha2Code
        self.alpha3Code = alpha3Code
        if topLevelDomain ==[]:     # Validating if the country has own web domain
            self.topLevelDomain = ""
        else:
            self.topLevelDomain = topLevelDomain[0]
        self.capital = capital
        self.population = population
        self.latlng = latlng
        self.distance_to_poland = distance_pl(latlng)   # Using distance_pl function to measure distance to Poland
        self.timezones = timezones
        all_lang = ""
        for lng in languages:       # Simple clarification for multiple languages
            all_lang = all_lang + f'{lng["name"]}, '
        self.languages = all_lang[:-2]
        if regionalBlocs == []:     # Validating if the Country in EU. Output as boolean
            self.regionalBlocs = False
        else:
            for bloc in regionalBlocs:
                if "EU" in bloc["acronym"]:
                    self.regionalBlocs = True
                else:
                    self.regionalBlocs = False
        if "symbol" in currencies[0]:       # Validating if the currency has it's own symbol
            self.symbol = currencies[0]["symbol"]
        else:
            self.symbol = ""
        self.curr_name = currencies[0]["name"]
        
        if currencies[0]["code"] in exchange_rates_dict:    # Using previously created dict. Appending the dict with a Currency code and value in PLN to minimize using the API
            self.curr_in_pln = exchange_rates_dict[currencies[0]["code"]]
        else:
            to_PLN = client.get_exchange_rates(currency=currencies[0]["code"])
            self.curr_in_pln = round(float(to_PLN["rates"]["PLN"]),2)
            exchange_rates_dict[currencies[0]["code"]] = round(float(to_PLN["rates"]["PLN"]),2)
        
        self.area = area

    
    
    @classmethod        # Class method for decoding JSON to objects using Country class
    def from_json(cls, json_string):
        json_dict = json.loads(json_string)
        return cls(**json_dict)

    def __repr__(self):     # Repr of an Object as a Country name
        return f'{self.name}'

if __name__ == '__main__':      # Making a csv file 
    filename = 'raport3.csv'
    items = []      # Items as a list of objects
    print("Converting data...")     
    for x in eu_json:
        items.append(Country.from_json(json.dumps(x))) # Most time consuming award winner - Appending Items list with Objects which are created using class method from JSON file

    print("Creating CSV file...")
    with open(filename, 'w',encoding="utf-8",newline='') as f:      # Writing in csv file and encoding it with "utf-8"
        writer = csv.writer(f)
        writer.writerow(["Country name","Native name","2 Code","3 Code","Web domain","Capital city","Population","Distance to Poland","Timezones","Languages","Is the Country in EU?","Area","Currency symbol","Currency name","Converted curr. to PLN","Extraction date"])     # Headers
        for item in items:  # For every object write proper attributes in one row
            writer.writerow([item.name ,item.nativeName,item.alpha2Code,item.alpha3Code,item.topLevelDomain,item.capital,item.population,item.distance_to_poland,item.timezones,item.languages,item.regionalBlocs,item.area,item.symbol,item.curr_name,item.curr_in_pln,datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")])
    
end = time.time()       # Ending time for measurement purposes

print(f"Done! It took me {round(end-start,2)} seconds")     # Printing duration of the function

Connecting...
Connected! Status code is: 200
Converting data...
Creating CSV file...
Done! It took me 4.93 seconds
